In [1]:
from PIL import Image 
import os
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pylab
import json

im_path = "" #path to current im folder
cropped_path = "" #path to cropped im folder 
resized_path = "" #path to resized im folder 

In [2]:
#only run this cell if you need to figure out the smallest dimension
smallest_dimen = 9000 #lol

for ii in os.listdir(im_path):
    file_name = os.path.basename(ii)
    old_path = os.path.join(im_path,file_name)
    image_opened = Image.open(old_path)
    height = image_opened.height
    width = image_opened.width
    
    if (width % 2) == 1:
        width = width - 1
    
    if (height % 2) == 1:
        height = height -1   
    
    if (height < width):
        dimen = height 
    
    if (width < height):
        dimen = width
    
    if (dimen < smallest_dimen):
        smallest_dimen = dimen
        print(smallest_dimen) #checking the smallest dimesion-- you can comment this out

1052
970
920


KeyboardInterrupt: 

In [3]:
#only run this cell if you already know the smallest dimension
smallest_dimen = 920

In [4]:
coco_annotation_file_path = "/home/u1/mec0104/Documents/Root AI/Merging datasets/AIRootsPNG/results_1/merged/annotations/merged.json" #path to annotation folder 
coco_annotations = COCO(annotation_file = coco_annotation_file_path)
cat_ids = coco_annotations.getCatIds()
print('COCO category ids:')
print(cat_ids)
cats = coco_annotations.loadCats(cat_ids)
names=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(names)))
image_ids = coco_annotations.getImgIds(catIds = cat_ids)

loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
COCO category ids:
[1]
COCO categories: 
plant root

[8548, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 

In [5]:
with open(coco_annotation_file_path) as json_file:
    data = json.load(json_file)
json_anns = data['annotations']
json_file.close()

In [6]:
for image_id in range(len(image_ids)):
    image = coco_annotations.loadImgs(image_ids[image_id])
    image = image[0]
    image_name = image['file_name']
    
    old_path = os.path.join(im_path,image_name)
    new_path = os.path.join(cropped_path,image_name)
    image_opened_2 = Image.open(old_path)
    height = image_opened_2.height
    width = image_opened_2.width 
    #print(height)
    #print(width)
    
    if (width % 2) == 1: 
        width = width - 1
     
    if (height % 2) == 1:
        height = height -1
                
    if(width<height):
        orig_height = height
        new_height = width 
        left = 0
        upper = 0.5*orig_height - 0.5*new_height
        right = width
        lower = 0.5*orig_height + 0.5*new_height 
        cropped_im = image_opened_2.crop((left, upper, right, lower))

        delta_x = 0 
        delta_y = (height - width)/2 
        scale = width/smallest_dimen

    if(height<width):
        orig_width = width
        new_width = height 
        left = 0.5*orig_width - 0.5*new_width
        upper = 0
        right = 0.5*orig_width + 0.5*new_width
        lower = height
        cropped_im = image_opened_2.crop((left, upper, right, lower))

        delta_x = (width - height)/2
        delta_y = 0 
        scale = height/smallest_dimen

    cropped_im.save(new_path)  
    
    old_path_2 = os.path.join(cropped_path,image_name)
    new_path_2 = os.path.join(resized_path,image_name)
    image_opened_3 = Image.open(old_path_2)
    image_opened_3.thumbnail((smallest_dimen, smallest_dimen))
    image_opened_3.save(new_path_2)
    
    annIds = coco_annotations.getAnnIds(imgIds=image['id'], catIds= cat_ids, iscrowd=None)
    anns = coco_annotations.loadAnns(annIds)
    
    for annotation_num in range(len(anns)):
        annotation = anns[annotation_num]
        
        bbox = annotation['bbox']
        bbox_width = bbox[2]/scale
        bbox_height =bbox[3]/scale
        bbox_x = (bbox[0] - delta_x)/scale
        bbox_y = (bbox [1] - delta_y)/scale
        bbox_new = [bbox_x, bbox_y, bbox_width, bbox_height]
    
        segmentation = annotation['segmentation']
        segmentation = segmentation[0]
        segmentation_new= [0]*len(segmentation)
        for coord in range(len(segmentation)):
            if coord % 2 == 0:
                segmentation_new[coord] = (segmentation[coord] - delta_x)/scale

            if coord % 2 == 1:
                segmentation_new[coord] = (segmentation[coord] - delta_y)/scale
        
        annotation['bbox'] = bbox_new
        annotation['segmentation'] = [segmentation_new]
        annotation_id = annotation['id']
        annotation_id_json = annotation_id-1
    
        json_anns[annotation_id_json] = annotation 

In [7]:
data['annotations'] = json_anns

In [8]:
images = data['images']
for image_num in range(len(images)):
    image_details = images[image_num]
    image_details['height'] = smallest_dimen
    image_details['width'] = smallest_dimen
    images[image_num] = image_details

data['images'] = images

In [9]:
result_file = open("./transformed_anns.json", "w+")
json.dump(data, result_file, indent = 4, sort_keys= True)

In [10]:
result_file.close()